# Collecting face images from fandom website

In [ ]:
import requests as re
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import os

## Collecting images from the Naruto fan wiki

In [ ]:
def collect_images(fandom_link, selector_overview, selector_character_page):
    # alphabetic character overview page -> character page -> character picture
    site_pointer = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","R","S","T","U","V","W","X","Y","Z","¡"]
    counter = 0
    # collect character links
    character_links = []
    for p in tqdm(site_pointer):
        soup = bs(re.get(fandom_link+p).text)
        for link in soup.select(selector_overview):
            character_links.append(fandom_link.split("/wiki")[0]+link.get("href"))
        
    #remove duplicates
    fandom_name = fandom_link.split(".fandom")[0].split("//")[1]
    try:
        os.mkdir(fandom_name)
    except:
        print("folder already exists")
    
    character_links = list(set(character_links))
    for character_link in tqdm(character_links):
        # Grab image links from character profile website
        img_links = [img_link.get("src").split("/revision")[0] for img_link in bs(re.get(character_link).text).select(selector_character_page)]
        for img_link in img_links:
            try:
                response = re.get(img_link)
            except:
                print("Error")
                continue
            #create folder using fandom name
            file = open(fandom_name+"/"+str(counter)+".png", "wb")
            file.write(response.content)
            file.close()
            counter+=1

# Avatar           
#fandom_link = "https://avatar.fandom.com/wiki/Category:Characters?from="
#selector_overview = ".category-page__member-link"
#selector_character_page = ".pi-image-thumbnail"

# Naruto 
fandom_link = "https://naruto.fandom.com/wiki/Category:Characters?from="
selector_overview = ".category-page__member-link"
selector_character_page = ".imagecell img"

collect_images(fandom_link, selector_overview, selector_character_page)

## Collecting images from Pinterest

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
import pickle

In [79]:
class pinterest_scraper:
    def __init__(self, username, password):
        self.driver = webdriver.Chrome()
        self.username = username
        self.password = password
        self.image_links = []
        self.keyword = ""
        self.image_links_collection = []
        
    def startup(self,keyword):
        self.keyword = keyword
        self.driver.get("https://www.pinterest.com/")
        time.sleep(2)
        self.login()
        self.driver.find_element_by_xpath("""//*[@id="searchBoxContainer"]/div/div/div[2]/input""").send_keys(keyword)
        self.driver.find_element_by_xpath("""//*[@id="searchBoxContainer"]/div/div/div[1]/input""").send_keys(Keys.RETURN)

    def login(self):
        self.driver.find_element_by_css_selector('.RCK').click()
        time.sleep(2)
        self.driver.find_element_by_id('email').send_keys(self.username)
        self.driver.find_element_by_id('password').send_keys(self.password)
        self.driver.find_element_by_css_selector('.SignupButton').click()
        time.sleep(4)

    def collect_image_links(self,num_images, save = False, link_file = None):
        if link_file is not None:
            with open(link_file, 'rb') as f:
                already_collected = pickle.load(f)
        else:
            already_collected = []
                
        while len(self.image_links) < num_images:
            t_0 = self.driver.execute_script("return document.body.scrollHeight")
            
            html = self.driver.page_source
            soup = bs(html)
            for image in soup.select(".L4E.MIw"):
                image_link = image.get("src")
                if "236x" in image_link:
                    image_link = image_link.replace("236x","originals")
                    if image_link not in already_collected:
                        self.image_links.append(image_link)
                    else:
                        print("link already collected")
            self.driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
            time.sleep(3)
            if t_0 == self.driver.execute_script("return document.body.scrollHeight"):
                print("breaking")
                break
        if save:
            with open("image_links_"+self.keyword+".pickle", 'wb') as handle:
                pickle.dump(self.image_links, handle)
        self.image_links_collection.extend(self.image_links)
        self.image_links = []
        return list(set(self.image_links))[:num_images]
    
    def download_images(self):
        try:
            os.mkdir("images_"+self.keyword)
        except:
                print("folder already exists")

        for counter, link in tqdm(enumerate(self.image_links)):
            response = re.get(link)
            
            soup = bs(response.content)
            if soup.select("code") != []:
                print("access denied")
                link = link.replace("originals","564x")
                response = re.get(link)

            file = open("images_"+self.keyword+"/"+str(counter)+".jpg", "wb")
            file.write(response.content)
            file.close()
            time.sleep(.2)
            counter+=1 

In [ ]:
username = ""
password = ""

ps = pinterest_scraper(username, password)
ps.startup("naruto characters")
ps.collect_image_links(1000, True)
ps.download_images()